<a href="https://colab.research.google.com/github/DeepikaBantu/Project-01/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas numpy scikit-learn xgboost streamlit joblib pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 90.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import joblib

# 1. Simulate IMD-style data
np.random.seed(42)
n = 1000
data = {
    'Temperature': np.random.uniform(15, 40, n),
    'Humidity': np.random.uniform(30, 100, n),
    'Pressure': np.random.uniform(950, 1020, n),
    'WindSpeed': np.random.uniform(0, 25, n),
    'RainfallToday': np.random.uniform(0, 50, n)
}
df = pd.DataFrame(data)
df['RainfallTomorrow'] = 0.3 * df['RainfallToday'] + 0.5 * (df['Humidity']/100) * np.random.uniform(0, 100, n)
df['RainTomorrow'] = (df['RainfallTomorrow'] > 20).astype(int)
df.to_csv('rainfall_data.csv', index=False)

# 2. Split features & targets
X = df[['Temperature', 'Humidity', 'Pressure', 'WindSpeed', 'RainfallToday']]
y = df['RainTomorrow']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Train Random Forest
rf_model = RandomForestClassifier(n_estimators=200, random_state=42)
rf_model.fit(X_train, y_train)

# 4. Train XGBoost
xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb_model.fit(X_train, y_train)

# 5. Evaluate
for name, model in [("Random Forest", rf_model), ("XGBoost", xgb_model)]:
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f"\n{name} Accuracy: {acc:.2f}")
    print(classification_report(y_test, y_pred))

# 6. Save models
joblib.dump(rf_model, "rf_model.pkl")
joblib.dump(xgb_model, "xgb_model.pkl")

print("\n✅ Models saved successfully!")


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [13:14:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



Random Forest Accuracy: 0.70
              precision    recall  f1-score   support

           0       0.70      0.56      0.62        86
           1       0.71      0.82      0.76       114

    accuracy                           0.70       200
   macro avg       0.70      0.69      0.69       200
weighted avg       0.70      0.70      0.70       200


XGBoost Accuracy: 0.66
              precision    recall  f1-score   support

           0       0.61      0.53      0.57        86
           1       0.68      0.75      0.71       114

    accuracy                           0.66       200
   macro avg       0.65      0.64      0.64       200
weighted avg       0.65      0.66      0.65       200


✅ Models saved successfully!


In [ ]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np

st.set_page_config(page_title="Rainfall Prediction System", page_icon="🌧️")

st.title("🌦️ Rainfall Prediction using ML (Random Forest & XGBoost)")
st.write("Enter today's weather parameters to predict if it will rain tomorrow and estimate rainfall amount.")

# Load models
rf_model = joblib.load("rf_model.pkl")
xgb_model = joblib.load("xgb_model.pkl")

# Input fields
temp = st.number_input("🌡️ Temperature (°C)", 0.0, 50.0, 25.0)
humidity = st.number_input("💧 Humidity (%)", 0.0, 100.0, 70.0)
pressure = st.number_input("🌬️ Pressure (hPa)", 900.0, 1100.0, 1000.0)
wind = st.number_input("🍃 Wind Speed (km/h)", 0.0, 50.0, 10.0)
rain_today = st.number_input("🌧️ Rainfall Today (mm)", 0.0, 100.0, 10.0)

model_choice = st.radio("Choose Model", ["Random Forest", "XGBoost"])

if st.button("🔍 Predict"):
    X_input = np.array([[temp, humidity, pressure, wind, rain_today]])

    if model_choice == "Random Forest":
        pred = rf_model.predict(X_input)[0]
    else:
        pred = xgb_model.predict(X_input)[0]

    # Simple rainfall estimation
    rainfall_amount = np.random.uniform(0, 100) if pred == 1 else np.random.uniform(0, 10)

    st.subheader(f"🌤️ Prediction Result: {'Rain Tomorrow ☔' if pred==1 else 'No Rain 🌞'}")
    st.write(f"💦 Estimated Rainfall: **{rainfall_amount:.2f} mm**")

    if rainfall_amount > 50:
        st.error("⚠️ Heavy Rain Alert! Please take necessary precautions.")
    elif pred == 1:
        st.warning("🌧️ Light to Moderate Rain Expected.")
    else:
        st.success("🌞 Clear weather likely tomorrow.")


2025-10-05 13:14:56.236 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.238 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.239 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.240 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-05 13:14:56.292 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!rm -rf /root/.ngrok2 /root/.config/ngrok


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# ⚠️ Replace YOUR_TOKEN_HERE with your ngrok authtoken
!ngrok config add-authtoken 33eGrIOXLuUFeB8T7279DRxqgIm_2sceHXQAmectS8Jw3sK6Y


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!pip install pyngrok
from pyngrok import ngrok

# ⚠️ Replace YOUR_TOKEN_HERE with your ngrok authtoken
!ngrok config add-authtoken 33eGrIOXLuUFeB8T7279DRxqgIm_2sceHXQAmectS8Jw3sK6Y


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("33eGrIOXLuUFeB8T7279DRxqgIm_2sceHXQAmectS8Jw3sK6Y")


In [ ]:
public_url = ngrok.connect(8501)
print("🌐 Public URL:", public_url)


🌐 Public URL: NgrokTunnel: "https://issuably-fathomless-adalberto.ngrok-free.dev" -> "http://localhost:8501"


In [ ]:
!streamlit run app.py &>/dev/null &
